## Imports

In [1]:
import os
from pathlib import Path

import numpy as np
import json

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "4"
import tensorflow as tf

tf.get_logger().setLevel("ERROR")


In [ ]:
import models.bert_learned_pooler as bert_learned_pooler

dicts = {}
for i in (0, 20, 40, 60, 80, 1, 2, 3, 4):
    bert_model = bert_learned_pooler.create_learned_pooler(i)
    bert_model.load_weights(
        f"./models/learned_pooler_epochs_{i}/training_checkpoints/ckpt_0001.ckpt"
    )
    w = bert_model.weights[-4].numpy()
    dicts[i] = w.tolist()
    del bert_model

with open("./learned_pooler_weights.json", "w") as f:
    json.dump(dicts, f)


In [4]:
with open("./learned_pooler_weights.json", "w") as f:
    for k, a in dicts.items():
        dicts[k] = a.tolist()
    json.dump(dicts, f)


In [7]:
dicts = json.load(open("./learned_pooler_weights.json", "r"))

for i, w in dicts.items():
    print(f"\nModel {i}")
    softmax_w = np.exp(w) / np.sum(np.exp(w), axis=0)
    for ind, weight in enumerate(softmax_w):
        print(f"layer: {ind}, weight: {weight:.4f}")



Model 0
layer: 0, weight: 0.0103
layer: 1, weight: 0.0096
layer: 2, weight: 0.0096
layer: 3, weight: 0.0088
layer: 4, weight: 0.0097
layer: 5, weight: 0.0083
layer: 6, weight: 0.0082
layer: 7, weight: 0.0093
layer: 8, weight: 0.0088
layer: 9, weight: 0.0085
layer: 10, weight: 0.0094
layer: 11, weight: 0.0196
layer: 12, weight: 0.0203
layer: 13, weight: 0.0347
layer: 14, weight: 0.0566
layer: 15, weight: 0.0745
layer: 16, weight: 0.0638
layer: 17, weight: 0.1556
layer: 18, weight: 0.1535
layer: 19, weight: 0.1168
layer: 20, weight: 0.0942
layer: 21, weight: 0.0396
layer: 22, weight: 0.0283
layer: 23, weight: 0.0255
layer: 24, weight: 0.0167

Model 20
layer: 0, weight: 0.0304
layer: 1, weight: 0.0258
layer: 2, weight: 0.0265
layer: 3, weight: 0.0273
layer: 4, weight: 0.0301
layer: 5, weight: 0.0274
layer: 6, weight: 0.0265
layer: 7, weight: 0.0269
layer: 8, weight: 0.0276
layer: 9, weight: 0.0290
layer: 10, weight: 0.0264
layer: 11, weight: 0.0273
layer: 12, weight: 0.0269
layer: 13, we

### evaluate specific model using squadv2 scripts

In [10]:
#
w = dicts["1"]
print(w)
top_12 = np.sort(np.asarray(w).argsort()[-12:][::-1])
print(top_12)


[-0.9234557747840881, -1.1578636169433594, -1.1638456583023071, -1.1462748050689697, -1.0800492763519287, -1.1886208057403564, -1.2443065643310547, -1.2031008005142212, -1.175255298614502, -1.0855493545532227, -0.9846654534339905, -0.9153114557266235, -0.7900750637054443, -0.5941738486289978, -0.7182539701461792, -0.8355004787445068, -0.8745389580726624, -0.9308702349662781, -0.8830348253250122, -0.5427570343017578, -0.2978352904319763, 0.18404620885849, 0.5236477851867676, 1.056945562362671, 2.3596701622009277]
[12 13 14 15 16 18 19 20 21 22 23 24]


In [11]:
# weights of the softmax
softmax_w = [np.exp(i) for i in w]
print(softmax_w / sum(softmax_w))


[0.01599765 0.01265477 0.01257929 0.01280228 0.01367882 0.01227147
 0.0116068  0.01209506 0.01243658 0.01360379 0.01504781 0.01612847
 0.01828028 0.02223625 0.01964148 0.01746846 0.01679966 0.01587948
 0.01665754 0.02340947 0.0299061  0.0484215  0.06800257 0.11591339
 0.42648103]


Get the top 12 absolute weights from layer_to_weight

### Extract the weights for all 25 hidden layers